In [1]:
# Test data: Never touched during training / model development, used for evaluating the final model
!wget https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/data/test.tsv

#saved model
!wget https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/saved_models/Adamax90.h5

# Load pretrained embeddings
!wget -nc https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

# Give -n argument so that a possible existing file isn't overwritten 
!unzip -n wiki-news-300d-1M.vec.zip

--2020-05-12 16:29:19--  https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1788466 (1.7M) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>]   1.71M  --.-KB/s    in 0.07s   

2020-05-12 16:29:19 (23.3 MB/s) - ‘test.tsv’ saved [1788466/1788466]

--2020-05-12 16:29:20--  https://raw.githubusercontent.com/sainvo/DeepLearning_NER/master/saved_models/Adamax90.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60228624 (57M) [application/o

In [2]:
import sys 
import csv

csv.field_size_limit(sys.maxsize)

131072

In [0]:
token = {"word":"","entity_label":""}

def read_ontonotes(tsv_file): # 
    current_sent = [] # list of (word,label) lists
    with open(tsv_file) as f:
        tsvreader = csv.reader(f, delimiter= '\n')
        for line in tsvreader:
            #print(line)
            if not line:
                if current_sent:
                    yield current_sent
                    current_sent=[]
                continue
            current_sent.append(line) 
        else:
            if current_sent:
                yield current_sent

full_test_data = list(read_ontonotes('test.tsv'))
#size_ts = int(len(full_test_data)/2)
#print(size_ts)
test_data_sample = full_test_data[:500]

In [6]:
import re
#regex for empty space chars, \t \n
tab = re.compile('[\t]')

def clean(list):
    clean_data =[]
    for sent in list:
        clean_list = []
        for item in sent:
            str = ''.join(item)
            #match_nl = re.match(r"\n", str)
            #print(match_nl)
            count_tab =  re.findall(r"\t", str)
            #print(count_tab)
            if len(count_tab) == 1: 
                item = re.split("\t", str)
                if item[0] != '.':
                    clean_list.append(item)
            elif len(count_tab) > 1:
                item = re.split("\n", str)
                #print(item)
                for i in range(len(item)):
                    #print(item[i])
                    if i == 0 or i == len(item)-1:
                        item[i] = '"'+item[i]
                        item[i] = re.split("\t", item[i])
                        #print(item[i])
                    else:
                        item[i] = re.split("\t", item[i])
                        #print(item[i])
                    clean_list.append(item[i])
        clean_data.append(clean_list)        
    return clean_data

test_data_clean = clean(test_data_sample)
print(len(test_data_clean))
for item in test_data_clean[:3]:
    print(item)

500
[['Powerful', 'B-WORK_OF_ART'], ['Tools', 'I-WORK_OF_ART'], ['for', 'I-WORK_OF_ART'], ['Biotechnology', 'I-WORK_OF_ART'], ['-', 'I-WORK_OF_ART'], ['Biochips', 'I-WORK_OF_ART']]
[['(', 'O'], ['Chang', 'B-PERSON'], ['Chiung', 'I-PERSON'], ['-', 'I-PERSON'], ['fang', 'I-PERSON'], ['/', 'O'], ['photos', 'O'], ['by', 'O'], ['Hsueh', 'B-PERSON'], ['Chi', 'I-PERSON'], ['-', 'I-PERSON'], ['kuang', 'I-PERSON'], ['/', 'O'], ['tr.', 'O'], ['by', 'O'], ['Robert', 'B-PERSON'], ['Taylor', 'I-PERSON'], [')', 'O']]
[['The', 'O'], ['enterovirus', 'O'], ['detection', 'O'], ['biochip', 'O'], ['developed', 'O'], ['by', 'O'], ['DR.', 'B-ORG'], ['Chip', 'I-ORG'], ['Biotechnology', 'I-ORG'], ['takes', 'O'], ['only', 'B-TIME'], ['six', 'I-TIME'], ['hours', 'I-TIME'], ['to', 'O'], ['give', 'O'], ['hospitals', 'O'], ['the', 'O'], ['answer', 'O'], ['to', 'O'], ['whether', 'O'], ['a', 'O'], ['sample', 'O'], ['contains', 'O'], ['enterovirus', 'O'], [',', 'O'], ['and', 'O'], ['if', 'O'], ['it', 'O'], ['is', 'O'

In [8]:
# shape into dicts per sentence

def reshape_sent2dicts(f):
    data_dict = []
    for item in f: # list of lists (tokens)
        #print(item)
        sent_text= [] 
        sent_tags = []
        for token in item:
            if len(token) ==2:
                sent_text.append(token[0])
                sent_tags.append(token[1])
        sent_dict = {'text':sent_text,'tags':sent_tags }
        #print(sent_dict['text'])
        #print(sent_dict['tags'])
        data_dict.append(sent_dict)
    return data_dict

test_data_sent = list(reshape_sent2dicts(test_data_clean[:30000]))

print(test_data_sent[:3])

[{'text': ['Powerful', 'Tools', 'for', 'Biotechnology', '-', 'Biochips'], 'tags': ['B-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART', 'I-WORK_OF_ART']}, {'text': ['(', 'Chang', 'Chiung', '-', 'fang', '/', 'photos', 'by', 'Hsueh', 'Chi', '-', 'kuang', '/', 'tr.', 'by', 'Robert', 'Taylor', ')'], 'tags': ['O', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'I-PERSON', 'I-PERSON', 'O', 'O', 'O', 'B-PERSON', 'I-PERSON', 'O']}, {'text': ['The', 'enterovirus', 'detection', 'biochip', 'developed', 'by', 'DR.', 'Chip', 'Biotechnology', 'takes', 'only', 'six', 'hours', 'to', 'give', 'hospitals', 'the', 'answer', 'to', 'whether', 'a', 'sample', 'contains', 'enterovirus', ',', 'and', 'if', 'it', 'is', 'the', 'deadly', 'strain', 'Entero', '71'], 'tags': ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O

In [9]:
import random
import numpy

random.seed(123)
random.shuffle(test_data_sent)
print(type(test_data_sent))
print(type(test_data_sent[0]))

test_texts=[i["text"] for i in test_data_sent]
test_labels=[i["tags"] for i in test_data_sent]

#print(type(train_texts))
#print(type(train_texts[0]))

print('Text: ', test_texts[:4])
print('Labels: ', test_labels[:4])

<class 'list'>
<class 'dict'>
Text:  [['Do', 'I', 'like', 'the', 'person', 'I', 'see', 'in', 'the', 'mirror', '?'], ['Ugly', 'duckling', 'no', 'more'], ['His', 'wife', 'was', 'a', 'universally', 'admired', 'beauty', ',', 'the', 'envy', 'of', 'all'], ['The', 'enterovirus', 'detection', 'biochip', 'developed', 'by', 'DR.', 'Chip', 'Biotechnology', 'takes', 'only', 'six', 'hours', 'to', 'give', 'hospitals', 'the', 'answer', 'to', 'whether', 'a', 'sample', 'contains', 'enterovirus', ',', 'and', 'if', 'it', 'is', 'the', 'deadly', 'strain', 'Entero', '71']]
Labels:  [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [10]:
from gensim.models import KeyedVectors

vector_model = KeyedVectors.load_word2vec_format("wiki-news-300d-1M.vec", binary=False, limit=50000)


# sort based on the index to make sure they are in the correct order
words = [k for k, v in sorted(vector_model.vocab.items(), key=lambda x: x[1].index)]
print("Words from embedding model:", len(words))
print("First 50 words:", words[:50])

# Normalize the vectors to unit length
print("Before normalization:", vector_model.get_vector("in")[:10])
vector_model.init_sims(replace=True)
print("After normalization:", vector_model.get_vector("in")[:10])

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Words from embedding model: 50000
First 50 words: [',', 'the', '.', 'and', 'of', 'to', 'in', 'a', '"', ':', ')', 'that', '(', 'is', 'for', 'on', '*', 'with', 'as', 'it', 'The', 'or', 'was', "'", "'s", 'by', 'from', 'at', 'I', 'this', 'you', '/', 'are', '=', 'not', '-', 'have', '?', 'be', 'which', ';', 'all', 'his', 'has', 'one', 'their', 'about', 'but', 'an', '|']
Before normalization: [-0.0234 -0.0268 -0.0838  0.0386 -0.0321  0.0628  0.0281 -0.0252  0.0269
 -0.0063]
After normalization: [-0.0163762  -0.01875564 -0.05864638  0.02701372 -0.02246478  0.04394979
  0.01966543 -0.0176359   0.01882563 -0.00440898]


In [11]:
# Build vocabulary mappings

# Zero is used for padding in Keras, prevent using it for a normal word.
# Also reserve an index for out-of-vocabulary items.
vocabulary={
    "<PAD>": 0,
    "<OOV>": 1
}

for word in words: # These are words from the word2vec model
    vocabulary.setdefault(word, len(vocabulary))

print("Words in vocabulary:",len(vocabulary))
inv_vocabulary = { value: key for key, value in vocabulary.items() } # invert the dictionary


# Embedding matrix
def load_pretrained_embeddings(vocab, embedding_model):
    """ vocab: vocabulary from our data vectorizer, embedding_model: model loaded with gensim """
    pretrained_embeddings = numpy.random.uniform(low=-0.05, high=0.05, size=(len(vocab)-1,embedding_model.vectors.shape[1]))
    pretrained_embeddings = numpy.vstack((numpy.zeros(shape=(1,embedding_model.vectors.shape[1])), pretrained_embeddings))
    found=0
    for word,idx in vocab.items():
        if word in embedding_model.vocab:
            pretrained_embeddings[idx]=embedding_model.get_vector(word)
            found+=1
            
    print("Found pretrained vectors for {found} words.".format(found=found))
    return pretrained_embeddings

pretrained=load_pretrained_embeddings(vocabulary, vector_model)

Words in vocabulary: 50002
Found pretrained vectors for 50000 words.


In [12]:
#Labels
from pprint import pprint

not_letter = re.compile(r'[^a-zA-Z]')
# Label mappings
# 1) gather a set of unique labels
label_set = set()
for sentence_labels in test_labels: #loops over sentences 
    #print(sentence_labels)
    for label in sentence_labels: #loops over labels in one sentence
       # match = not_letter.match(label)
        #if match or label== 'O':
        #    break
        #else:    
        label_set.add(label)

# 2) index these
label_map = {}
for index, label in enumerate(label_set):
    label_map[label]=index
    
pprint(label_map)

{'B-CARDINAL': 14,
 'B-DATE': 8,
 'B-EVENT': 16,
 'B-FAC': 13,
 'B-GPE': 19,
 'B-LANGUAGE': 21,
 'B-LOC': 7,
 'B-MONEY': 22,
 'B-NORP': 28,
 'B-ORDINAL': 5,
 'B-ORG': 3,
 'B-PERCENT': 23,
 'B-PERSON': 10,
 'B-QUANTITY': 17,
 'B-TIME': 26,
 'B-WORK_OF_ART': 18,
 'I-CARDINAL': 11,
 'I-DATE': 4,
 'I-EVENT': 2,
 'I-FAC': 9,
 'I-GPE': 12,
 'I-LOC': 6,
 'I-MONEY': 27,
 'I-NORP': 15,
 'I-ORG': 20,
 'I-PERCENT': 24,
 'I-PERSON': 1,
 'I-QUANTITY': 30,
 'I-TIME': 29,
 'I-WORK_OF_ART': 25,
 'O': 0}


In [13]:
# vectorize the labels
def label_vectorizer(train_labels,label_map):
    vectorized_labels = []
    for label in test_labels:
        vectorized_example_label = []
        for token in label:
            if token in label_map:
                vectorized_example_label.append(label_map[token])
        vectorized_labels.append(vectorized_example_label)
    vectorized_labels = numpy.array(vectorized_labels)
    return vectorized_labels
        

vectorized_labels = label_vectorizer(test_labels,label_map)
#validation_vectorized_labels = label_vectorizer(dev_labels,label_map)

pprint(vectorized_labels[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
## vectorization of the text
def text_vectorizer(vocab, test_texts):
    vectorized_data = [] # turn text into numbers based on our vocabulary mapping
    sentence_lengths = [] # Number of tokens in each sentence
    
    for i, one_example in enumerate(test_texts):
        vectorized_example = []
        for word in one_example:
            vectorized_example.append(vocab.get(word, 1)) # 1 is our index for out-of-vocabulary tokens

        vectorized_data.append(vectorized_example)     
        sentence_lengths.append(len(one_example))
        
    vectorized_data = numpy.array(vectorized_data) # turn python list into numpy array
    
    return vectorized_data, sentence_lengths

vectorized_data, lengths=text_vectorizer(vocabulary, test_texts)
#validation_vectorized_data, validation_lengths=text_vectorizer(vocabulary, dev_texts)

pprint(test_texts[0])
pprint(vectorized_data[0])

['Do', 'I', 'like', 'the', 'person', 'I', 'see', 'in', 'the', 'mirror', '?']
[1955, 30, 68, 3, 363, 30, 138, 8, 3, 5563, 39]


In [21]:
# padding for tensor
import tensorflow as tf
### Only needed for me, not to block the whole GPU, you don't need this stuff
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.3
#set_session(tf.Session(config=config))
### ---end of weird stuff

from keras.preprocessing.sequence import pad_sequences
print("Old shape:", vectorized_data.shape)
vectorized_data_padded=pad_sequences(vectorized_data, padding='pre', maxlen=max(lengths))
print("New shape:", vectorized_data_padded.shape)
print("First example:")
print( vectorized_data_padded[0])
# Even with the sparse output format, the shape has to be similar to the one-hot encoding
vectorized_labels_padded=numpy.expand_dims(pad_sequences(vectorized_labels, padding='pre', maxlen=max(lengths)), -1)
print("Padded labels shape:", vectorized_labels_padded.shape)
pprint(label_map)
print("First example labels:")
pprint(vectorized_labels_padded[0])

weights = numpy.copy(vectorized_data_padded)
weights[weights > 0] = 1
print("First weight vector:")
print( weights[0])

Using TensorFlow backend.


Old shape: (500,)
New shape: (500, 76)
First example:
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0 1955   30   68    3  363
   30  138    8    3 5563   39]
Padded labels shape: (500, 76, 1)
{'B-CARDINAL': 14,
 'B-DATE': 8,
 'B-EVENT': 16,
 'B-FAC': 13,
 'B-GPE': 19,
 'B-LANGUAGE': 21,
 'B-LOC': 7,
 'B-MONEY': 22,
 'B-NORP': 28,
 'B-ORDINAL': 5,
 'B-ORG': 3,
 'B-PERCENT': 23,
 'B-PERSON': 10,
 'B-QUANTITY': 17,
 'B-TIME': 26,
 'B-WORK_OF_ART': 18,
 'I-CARDINAL': 11,
 'I-DATE': 4,
 'I-EVENT': 2,
 'I-FAC': 9,
 'I-GPE': 12,
 'I-LOC': 6,
 'I-MONEY': 27,
 'I-NORP': 15,
 'I-ORG': 20,
 'I-PERCENT': 24,
 'I-PERSON': 1,
 'I-QUANTITY': 30,
 'I-TIME': 29,
 'I-WORK_OF_ART': 25,
 'O': 0}
First example labels:
array([[0]

In [0]:
from tensorflow.keras.models import load_model

model_EL = load_model('Adamax90.h5')

In [0]:
model_EL.predict_classes(test)